In [1]:
import polars as pl

In [36]:
df = pl.read_ndjson('./experiments/base/tests.ndjson')
df = df.explode('predicted_p_values', 'true_p_values')
df = df.with_columns(experiment_type=pl.col('name').str.slice(0,3))
len(df)

1684

In [37]:
df.plot.scatter(
    x='predicted_p_values',
    y='true_p_values',
    alpha=0.7,
    ylim=(-0.1,1.1),
    xlim=(-0.1,1.1),
    height=400,
    width=400,
    #by='name', width=600,
    groupby=['experiment_type', 'num_clients', 'num_samples']
    )

BokehModel(combine_events=True, render_bundle={'docs_json': {'2ea71da9-12c6-45d9-a422-d4cd79652162': {'version…

In [38]:
_df = df
_df = _df.group_by('name', 'experiment_type', 'num_clients', 'num_samples') \
    .agg(pl.corr('predicted_p_values', 'true_p_values')) \
    .rename({'predicted_p_values': 'p_value_correlation'}) 
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='p_value_correlation',
                                  alpha=0.6,
                                  by='name',
                                  groupby=['experiment_type','num_clients'],
                                  ylim=(0.5,1.001)
                                  )

BokehModel(combine_events=True, render_bundle={'docs_json': {'00860a50-f76d-4559-a08b-5d262691688f': {'version…

In [42]:
alpha = 0.05

_df = df
_df = _df.with_columns(
    tp=(pl.col('predicted_p_values') < alpha) & (pl.col('true_p_values') < alpha),
    tn=(pl.col('predicted_p_values') > alpha) & (pl.col('true_p_values') > alpha),
    fp=(pl.col('predicted_p_values') < alpha) & (pl.col('true_p_values') > alpha),
    fn=(pl.col('predicted_p_values') > alpha) & (pl.col('true_p_values') < alpha),
)

_df = _df.group_by('name', 'experiment_type', 'num_clients', 'num_samples').agg((pl.col('tp')+pl.col('tn')).mean().alias('accuracy'))
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='accuracy',
                                  alpha=0.6,
                                  by='name',
                                  groupby=['experiment_type','num_clients'],
                                  ylim=(0.5,1.001)
                                  )

BokehModel(combine_events=True, render_bundle={'docs_json': {'1b24183b-b6ca-4c25-90d3-4fd08b171dc0': {'version…